In [58]:
import pandas as pd
import numpy as np
import csv
import zipfile
import re

import random
import matplotlib as mpl
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from scipy.misc import imread
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from wordcloud import WordCloud, STOPWORDS


%matplotlib inline
filepath = "../resources/food.csv.zip"
zf = zipfile.ZipFile(filepath)
food = pd.read_csv(zf.open('food.csv'), parse_dates=True, dtype=object, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows',10)

food

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000024600,http://world-en.openfoodfacts.org/product/0000...,date-limite-app,1434530704,2015-06-17T08:45:04Z,1434535914,2015-06-17T10:11:54Z,Filet de bœuf,NaN,2.46 kg,plastique,plastique,NaN,NaN,Filet de bœuf,fr:filet-de-boeuf,fr:Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:

In [59]:
def printCategoryInfo(dataframe, min_values=2):
    with pd.option_context('display.max_rows', 99999, 'display.max_columns', 1):
        print("Product names:")
        printWordOccurences(str(dataframe["product_name"]), min_values)
        print("-----------")
        print("Categories_en:")
        printWordOccurences(str(dataframe["categories_en"]), min_values)
        print("-----------")
        print("Categories_tags:")
        printWordOccurences(str(dataframe["categories_tags"]), min_values)
        print("-----------")
        print("Categories:")
        printWordOccurences(str(dataframe["categories"]), min_values)

In [60]:
def grey_color(word, font_size, position, orientation, random_state=None, **kwargs):
    return 'hsl(0, 0%%, %d%%)' % random.randint(50, 100)

def draw_wordcloud(text):
    # mpl.style.use('dark')
    limit = 1000
    infosize = 12

    fontcolor='#fafafa'
    bgcolor = '#000000'
    # english_stopwords = set(stopwords.words('english')) | STOPWORDS | ENGLISH_STOP_WORDS
    
    wordcloud = WordCloud(
        max_words=limit,
        # stopwords=english_stopwords,
        # mask=imread('img/sherlock-holmes-silhouette.png'),
        background_color=bgcolor,
        #font_path=font
    ).generate(text)
    
    fig = plt.figure()
    fig.set_figwidth(14)
    fig.set_figheight(18)

    plt.imshow(wordcloud.recolor(color_func=grey_color, random_state=3))
    #plt.title(title, color=fontcolor, size=30, y=1.01)
    #plt.annotate(footer, xy=(0, -.025), xycoords='axes fraction', fontsize=infosize, color=fontcolor)
    plt.axis('off')
    plt.show()

In [61]:
# from stackoverflow
def getWordScores(text):
    scores = {}
    for word in text.strip().split():
        word = word.lower()
        scores[word] = scores.get(word,0) + 1
    return scores

def printWordOccurences(text, min_word_count=0):
    import operator
    text = getWordScores(text)
    sorted_text = sorted(text.items(), key=operator.itemgetter(1))
    for i in sorted_text:
        if int(i[1]) >= min_word_count:
            print(i)

In [62]:
# get a list of all candy tags and combinations of candy tags, get all foods who's tags are in that list.
# bad approach because it's slow and if something contains candy tags + non-candy tags (e.g. 'desserts, water')
# it falls through the cracks. Sadly I can't figure out how to do
# "for candy_tag in candy_tags: get items where candy_tag in food['categories_tags']"

from itertools import combinations

candy_tags = ["en:sugary-snacks", "en:desserts", "en:candies", "en:chocolates", "en:pies", "en:puddings", 
 "en:biscuits-and-cakes", "en:confectioneries", "en:chocolate-chip-cookies", "en:chocolate-block", 
 "en:jelly-crystals", "en:carrot-cake", "en:chocolate-covered-muesli-bar"]
augmented_tags = candy_tags

for i in range(2, 4):
    augmented_tags += [",".join(map(str, comb)) for comb in combinations(candy_tags, i)]

#augmented_tags
tagged_candies = food[food['categories_tags'].isin(augmented_tags)]

#tagged_candies = [item for item in food if not set(augmented_tags).isdisjoint(item['categories_tags'])]
                           
tagged_candies

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
193,0009542018085,http://world-en.openfoodfacts.org/product/0009...,stephane,1438277425,2015-07-30T17:30:25Z,1438566765,2015-08-03T01:52:45Z,NaN,NaN,3.5 OZ (100g),NaN,NaN,NaN,NaN,chocolate,"en:sugary-snacks,en:chocolates","Sugary snacks,Chocolates",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,United States,NaN,NaN,NaN,NaN,NaN,NaN,4 pieces (40g),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e,Sugary snacks,Chocolate products,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",en:sugary-snacks,Sugary snacks,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,2200,1150,30,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.025,55,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,5,NaN,NaN,NaN,0.159,0.0625,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,0.0009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,23
293,00116954,http://world-en.openfoodfacts.org/product/0011...,tacinte,1408885358,2014-08-24T13:02:38Z,1424686440,2015-02-23T10:14:00Z,Swiss Dark Chocolate,Swiss Extra Dark Chocolate,100g,"Carton,Card","carton,card","Sainsbury's,sainsbury's taste the difference","sai

In [63]:
food_not_tagged_as_candy = food[~food['categories_tags'].isin(augmented_tags)]

food_not_tagged_as_candy

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"en:to-be-completed, en:nutrition-facts-to-be-c...","en:to-be-completed,en:nutrition-facts-to-be-co...","To be completed,Nutrition facts to be complete...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000024600,http://world-en.openfoodfacts.org/product/0000...,date-limite-app,1434530704,2015-06-17T08:45:04Z,1434535914,2015-06-17T10:11:54Z,Filet de bœuf,NaN,2.46 kg,plastique,plastique,NaN,NaN,Filet de bœuf,fr:filet-de-boeuf,fr:Filet-de-boeuf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,France,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,unknown,"en:to-be-completed, en:

In [64]:
candies_around_50_carbs = tagged_candies[tagged_candies['carbohydrates_100g'].astype(float) > 47]
candies_around_50_carbs = candies_around_50_carbs[candies_around_50_carbs['carbohydrates_100g'].astype(float) < 57]

candies_around_50_carbs

#candies_below = tagged_candies[tagged_candies['carbohydrates_100g'].astype(float) < 20]
#candies_below
#candies_above = tagged_candies[tagged_candies['carbohydrates_100g'].astype(float) > 65]
#candies_above

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
193,0009542018085,http://world-en.openfoodfacts.org/product/0009...,stephane,1438277425,2015-07-30T17:30:25Z,1438566765,2015-08-03T01:52:45Z,NaN,NaN,3.5 OZ (100g),NaN,NaN,NaN,NaN,chocolate,"en:sugary-snacks,en:chocolates","Sugary snacks,Chocolates",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States,en:united-states,United States,NaN,NaN,NaN,NaN,NaN,NaN,4 pieces (40g),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,e,Sugary snacks,Chocolate products,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",en:sugary-snacks,Sugary snacks,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,2200,1150,30,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0.025,55,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.5,5,NaN,NaN,NaN,0.159,0.0625,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.15,NaN,0.0009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23,23
1268,0034000440887,http://world-en.openfoodfacts.org/product/0034...,kalepom,1454428697,2016-02-02T15:58:17Z,1462023368,2016-04-30T13:36:08Z,Reese's,Peanut butter in a chocolate flavoured coating,51 g,Plastique,plastique,"The Hershey Company,Reese's","the-hershey-com

In [65]:
candies_around_50_sugar = tagged_candies[tagged_candies['sugars_100g'].astype(float) > 45]
candies_around_50_sugar = candies_around_50_carbs[candies_around_50_carbs['sugars_100g'].astype(float) < 55]

candies_around_50_sugar

#candies_below = tagged_candies[tagged_candies['sugars_100g'].astype(float) < 30]
#candies_below
candies_above = tagged_candies[tagged_candies['sugars_100g'].astype(float) > 65]
candies_above

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
1768,0043000204337,http://world-en.openfoodfacts.org/product/0043...,stephane,1441263361,2015-09-03T06:56:01Z,1462246205,2016-05-03T03:30:05Z,Vanilla,Instant Pudding & Pie Filling,NaN,Card,card,Jell-o,jell-o,Pudding,"en:desserts,en:puddings","Desserts,Puddings",NaN,NaN,NaN,NaN,Kosher,en:kosher,Kosher,NaN,NaN,NaN,NaN,NaN,New York,NaN,United States,en:united-states,United States,"SUGAR, MODIFIED CORN STARCH, CONTAINS LESS THA...",NaN,NaN,NaN,NaN,NaN,25 g,NaN,4,[ sugar -> en:sugar ] [ modified-corn-starc...,"en:e450iii,en:e102,en:e110,en:e320","E450iii - Tetrasodium diphosphate,E102 - Tartr...",0,NaN,NaN,0,NaN,NaN,NaN,e,unknown,unknown,"en:to-be-completed, en:nutrition-facts-complet...","en:to-be-completed,en:nutrition-facts-complete...","To be completed,Nutrition facts completed,Ingr...",en:desserts,Desserts,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,1510,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,92,76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,3.56,1.4,NaN,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,24
1769,0043000204450,http://world-en.openfoodfacts.org/product/0043...,aaronasachimp,1462241673,2016-05-03T02:14:33Z,1462246151,2016-

In [66]:
candies_around_x_fats = tagged_candies[tagged_candies['fat_100g'].astype(float) < 5]
candies_around_x_fats = candies_around_x_fats[candies_around_x_fats['fat_100g'].astype(float) < 5]

#candies_around_x_fats = candies_around_x_fats[candies_around_x_fats['carbohydrates_100g'].astype(float) > 30]


candies_around_x_fats

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
781,0021000419074,http://world-en.openfoodfacts.org/product/0021...,tacinte,1421613892,2015-01-18T20:44:52Z,1439141748,2015-08-09T17:35:48Z,Snack Pack Pudding Vanilla,NaN,4 x 3.25 oz,plastic pot,plastic-pot,"Snack Pack,ConAgra foods","snack-pack,conagra-foods",Desserts,en:desserts,Desserts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Chicago,Illinois,USA",Kmart,United States,en:united-states,United States,"Water, nonfat milk, sugar, modified corn sta...",NaN,NaN,NaN,NaN,NaN,92 g,NaN,5,[ water -> en:water ] [ nonfat-milk -> en:n...,"en:e170,en:e481,en:e339ii,en:e102,en:e110","E170 - Calcium carbonate,E481 - Sodium stearoy...",0,NaN,NaN,0,NaN,NaN,NaN,c,unknown,unknown,"en:to-be-checked, en:complete, en:nutrition-fa...","en:to-be-checked,en:complete,en:nutrition-fact...","To be checked,Complete,Nutrition facts complet...",en:desserts,Desserts,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,454,NaN,3.26,1.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.09,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,19.6,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.09,1.09,NaN,NaN,NaN,0.304,0.12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0217,NaN,0.326,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4
1768,0043000204337,http://world-en.openfoodfacts.org/product/0043.

In [67]:
candies_around_x_sugar = tagged_candies[tagged_candies['sugars_100g'].astype(float) > 22]
candies_around_x_sugar = candies_around_x_sugar[candies_around_x_sugar['sugars_100g'].astype(float) < 30]

candies_around_x_sugar = candies_around_x_sugar[candies_around_x_sugar['carbohydrates_100g'].astype(float) > 30]


candies_around_x_sugar

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
293,00116954,http://world-en.openfoodfacts.org/product/0011...,tacinte,1408885358,2014-08-24T13:02:38Z,1424686440,2015-02-23T10:14:00Z,Swiss Dark Chocolate,Swiss Extra Dark Chocolate,100g,"Carton,Card","carton,card","Sainsbury's,sainsbury's taste the difference","sainsbury-s,sainsbury-s-taste-the-difference","Sugary snacks,Chocolates","en:sugary-snacks,en:chocolates","Sugary snacks,Chocolates",NaN,NaN,Switzerland,switzerland,"UTZ Certified,Taste tested by customers","en:sustainable-farming,en:utz-certified,en:tas...","Sustainable farming,UTZ Certified,Taste-tested...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,United Kingdom,en:united-kingdom,United Kingdom,"Cocoa Mass, Sugar, Cocoa Butter, Cocoa Powder,...",NaN,NaN,"Nuts,Milk,Soya","en:milk,en:nuts,en:soybeans","Milk,Nuts,Soybeans",NaN,NaN,0,[ cocoa-mass -> en:cocoa-mass ] [ mass -> e...,NaN,NaN,0,NaN,NaN,0,NaN,NaN,NaN,d,Sugary snacks,Chocolate products,"en:to-be-checked, en:complete, en:nutrition-fa...","en:to-be-checked,en:complete,en:nutrition-fact...","To be checked,Complete,Nutrition facts complet...",en:sugary-snacks,Sugary snacks,http://en.openfoodfacts.org/images/products/00...,http://en.openfoodfacts.org/images/products/00...,2433,NaN,45.6,27.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.4,1.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.3,26.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.6,7.6,NaN,NaN,NaN,

In [68]:
candies_around_x_carbs = tagged_candies[tagged_candies['carbohydrates_100g'].astype(float) > 22]
candies_around_x_carbs = candies_around_x_carbs[candies_around_x_carbs['carbohydrates_100g'].astype(float) < 30]

candies_around_x_carbs

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_en,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_en,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_en,ingredients_text,allergens,allergens_en,traces,traces_tags,traces_en,serving_size,no_nutriments,additives_n,additives,additives_tags,additives_en,ingredients_from_palm_oil_n,ingredients_from_palm_oil,ingredients_from_palm_oil_tags,ingredients_that_may_be_from_palm_oil_n,ingredients_that_may_be_from_palm_oil,ingredients_that_may_be_from_palm_oil_tags,nutrition_grade_uk,nutrition_grade_fr,pnns_groups_1,pnns_groups_2,states,states_tags,states_en,main_category,main_category_en,image_url,image_small_url,energy_100g,energy-from-fat_100g,fat_100g,saturated-fat_100g,-butyric-acid_100g,-caproic-acid_100g,-caprylic-acid_100g,-capric-acid_100g,-lauric-acid_100g,-myristic-acid_100g,-palmitic-acid_100g,-stearic-acid_100g,-arachidic-acid_100g,-behenic-acid_100g,-lignoceric-acid_100g,-cerotic-acid_100g,-montanic-acid_100g,-melissic-acid_100g,monounsaturated-fat_100g,polyunsaturated-fat_100g,omega-3-fat_100g,-alpha-linolenic-acid_100g,-eicosapentaenoic-acid_100g,-docosahexaenoic-acid_100g,omega-6-fat_100g,-linoleic-acid_100g,-arachidonic-acid_100g,-gamma-linolenic-acid_100g,-dihomo-gamma-linolenic-acid_100g,omega-9-fat_100g,-oleic-acid_100g,-elaidic-acid_100g,-gondoic-acid_100g,-mead-acid_100g,-erucic-acid_100g,-nervonic-acid_100g,trans-fat_100g,cholesterol_100g,carbohydrates_100g,sugars_100g,-sucrose_100g,-glucose_100g,-fructose_100g,-lactose_100g,-maltose_100g,-maltodextrins_100g,starch_100g,polyols_100g,fiber_100g,proteins_100g,casein_100g,serum-proteins_100g,nucleotides_100g,salt_100g,sodium_100g,alcohol_100g,vitamin-a_100g,beta-carotene_100g,vitamin-d_100g,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g
10579,20229551,http://world-en.openfoodfacts.org/product/2022...,aleene,1439659902,2015-08-15T17:31:42Z,1439660775,2015-08-15T17:46:15Z,Noir dégustation,Chocolat noir,125g,"papier,alu","papier,alu",J.D.Gross,j-d-gross,"Snacks sucrés,Chocolats","en:sugary-snacks,en:chocolates","Sugary snacks,Chocolates",Équateur,equateur,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Gap,France",Lidl,France,en:france,France,"pâte de cacao, sucre, cacoa maigre en poudre, ...",NaN,NaN,"fruits à coque,lait,soja","en:milk,en:nuts,en:soybeans","Milk,Nuts,Soybeans","12,5g",NaN,1,[ pate-de-cacao -> fr:pate-de-cacao ] [ pat...,en:e322,E322 - Lecithins,0,NaN,NaN,0,NaN,NaN,NaN,e,Sugary snacks,Chocolate products,"en:to-be-checked, en:complete, en:nutrition-fa...","en:to-be-checked,en:complete,en:nutrition-fact...","To be checked,Complete,Nutrition facts complet...",en:sugary-snacks,Sugary snacks,http://en.openfoodfacts.org/images/products/20...,http://en.openfoodfacts.org/images/products/20...,2274,NaN,42.8,26.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.6,16.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.2,NaN,NaN,NaN,0.03,0.0118110236220472,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,19
42781,3271598507357,http://worl

In [69]:
import plotly.plotly as py
import plotly.graph_objs as go
py.sign_in('Odin123', '5Ba7VLNrZsqtUXUatHiZ')

candy_sugar = tagged_candies["sugars_100g"].astype(float).mean()
others_sugar = food_not_tagged_as_candy["sugars_100g"].astype(float).mean()

candy_fat = tagged_candies["fat_100g"].astype(float).mean()
others_fat = food_not_tagged_as_candy["fat_100g"].astype(float).mean()

candy_energy = tagged_candies["energy_100g"].astype(float).mean() / 1000
others_energy = food_not_tagged_as_candy["energy_100g"].astype(float).mean() / 1000

candy_carbs = tagged_candies["carbohydrates_100g"].astype(float).mean()
others_carbs = food_not_tagged_as_candy["carbohydrates_100g"].astype(float).mean()

candy_salt = tagged_candies["salt_100g"].astype(float).mean()
others_salt = food_not_tagged_as_candy["salt_100g"].astype(float).mean()

candy_nutrition_score = tagged_candies["nutrition-score-fr_100g"].astype(float).mean()
others_nutrition_score = food_not_tagged_as_candy["nutrition-score-fr_100g"].astype(float).mean()

x_names = ["sugar", "fat", "energy (MJ)", "carbohydrates", "salt", "nutrition score"]
y_values_candy = [candy_sugar, candy_fat, candy_energy, candy_carbs, candy_salt, candy_nutrition_score]
y_values_others = [others_sugar, others_fat, others_energy, others_carbs, others_salt, others_nutrition_score]

trace1 = go.Bar(
    x=x_names,
    y=y_values_candy,
    name='Candy'
)
trace2 = go.Bar(
    x=x_names,
    y=y_values_others,
    name='Other Foods'
)

data = [trace1, trace2]
layout = go.Layout(
    #xaxis=dict(tickangle=-180),
    barmode='group',
    height=700,
    margin=go.Margin(
        l=50,
        r=50,
        b=150,
        t=100,
        pad=4
    ),
)

fig = go.Figure(data=data, layout=layout)
# comment out figure-creation to save API access volume
#py.iplot(fig, filename='candy_others_comparison_bars', bbox_inches='tight')

In [70]:
trace1 = go.Histogram(
    x=tagged_candies["salt_100g"].astype(float),
)

layout = go.Layout(
    title='Salt (in g) in Candy',
    xaxis=dict(
        title='salt in 100g (steps of 5)'
    ),
    yaxis=dict(
        title='Number of candy in interval'
    ),
    bargap=0.01
)
data = [trace1]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_salt_histo')

In [71]:
trace1 = go.Histogram(
    x=tagged_candies["sugars_100g"].astype(float),
)

layout = go.Layout(
    title='Sugar (in g) in Candy',
    xaxis=dict(
        title='sugar in 100g (steps of 5)'
    ),
    yaxis=dict(
        title='Number of candy in interval'
    ),
    bargap=0.01
)
data = [trace1]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_sugar_histo')

In [72]:
trace1 = go.Histogram(
    x=tagged_candies["fat_100g"].astype(float),
)

layout = go.Layout(
    title='Fat (in g) in Candy',
    xaxis=dict(
        title='fat in 100g'
    ),
    yaxis=dict(
        title='Number of candy in interval'
    ),
    bargap=0.01
)
data = [trace1]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_fat_histo')

In [73]:
trace1 = go.Histogram(
    x=tagged_candies["carbohydrates_100g"].astype(float),
)

layout = go.Layout(
    title='Carbohydrates (in g) in Candy',
    xaxis=dict(
        title='carbohydrates in 100g (steps of 5)'
    ),
    yaxis=dict(
        title='Number of candy in interval'
    ),
    bargap=0.01
)
data = [trace1]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_carbs_histo')

In [74]:
trace1 = go.Histogram(
    x=food_not_tagged_as_candy["carbohydrates_100g"].astype(float),
)

layout = go.Layout(
    title='Carbohydrates in Others',
    xaxis=dict(
        title='carbohydrates in 100g'
    ),
    yaxis=dict(
        title='Number of items in interval'
    ),
    bargap=0.01
)
data = [trace1]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='others_carbs_histo')

In [75]:
## WEEK 4 ##

trace = go.Scatter(
    x = tagged_candies["fat_100g"].astype(float),
    y = tagged_candies["sugars_100g"].astype(float),
    mode = 'markers'
)

layout = go.Layout(
    title= 'Sweets: Fat to Sugar ratio',
    xaxis= dict(
        range=[0, 100],
        title="Fat in g / 100g"
    ),
    yaxis=dict(
        range=[0, 100],
        title="Sugars in g / 100g"
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_fat_sugar_ratio_scatter')

In [76]:
trace = go.Scatter(
    x = food_not_tagged_as_candy["fat_100g"].astype(float),
    y = food_not_tagged_as_candy["sugars_100g"].astype(float),
    mode = 'markers'
)

layout = go.Layout(
    title= 'Non-sweets: Fat to Sugar ratio',
    xaxis= dict(
        range=[0, 100],
        title="Fat in g / 100g"
    ),
    yaxis=dict(
        range=[0, 100],
        title="Sugars in g / 100g"
    )
)

data = [trace]

#fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='non-candy_fat_sugar_ratio_scatter')

In [77]:
food_not_tagged_as_candy['fat/sugar'] = food_not_tagged_as_candy['fat_100g'].astype(float) / food_not_tagged_as_candy['sugars_100g'].astype(float)

trace = go.Scatter(
    x = food_not_tagged_as_candy['fat/sugar'],
    y = food_not_tagged_as_candy["salt_100g"].astype(float),
    mode = 'markers'
)

layout = go.Layout(
    title= 'Non-sweets: (Fat to Sugar) to salt ratio',
    xaxis= dict(
        range=[0, 100],
        title="(Fat to Sugar) in g / 100g"
    ),
    yaxis=dict(
        range=[0, 100],
        title="salt in g / 100g"
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='non-candy_fat-sugar_salt_ratio_scatter')

d:\Users\Odin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [78]:
# non-candy in the area where candy is on the same graph: chocolate[~1000], (fruit-based) beverages[~5000] sugary [~2000]
low_ratio_non_candy = food_not_tagged_as_candy[food_not_tagged_as_candy['fat/sugar'] <= 1]
low_ratio_non_candy = low_ratio_non_candy[low_ratio_non_candy["salt_100g"].astype(float) < 0.4]

#printCategoryInfo(low_ratio_non_candy, 200)

In [79]:
tagged_candies['fat/sugar'] = tagged_candies['fat_100g'].astype(float) / tagged_candies['sugars_100g'].astype(float)

trace = go.Scatter(
    x = tagged_candies['fat/sugar'],
    y = tagged_candies["salt_100g"].astype(float),
    mode = 'markers'
)

layout = go.Layout(
    title= 'Sweets: (Fat to Sugar) to salt ratio',
    xaxis= dict(
        range=[0, 100],
        title="(Fat to Sugar) in g / 100g"
    ),
    yaxis=dict(
        range=[0, 100],
        title="salt in g / 100g"
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_fat-sugar_salt_ratio_scatter')

d:\Users\Odin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [80]:
fat_sugar_outlier = tagged_candies[tagged_candies['salt_100g'].astype(float) > 0.47]
#print(fat_sugar_outlier)

In [81]:
# outlier to the right
candies_at_x_sugar_fat = tagged_candies[tagged_candies['sugars_100g'].astype(float) == 11.7]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['fat_100g'].astype(float) == 67]

#candies_at_x_sugar_fat # Quernons d'ardoise Chocolat et nougatine


In [82]:
trace = go.Scatter(
    x = food_not_tagged_as_candy['fat/sugar'],
    y = food_not_tagged_as_candy["energy_100g"].astype(float),
    mode = 'markers'
)

layout = go.Layout(
    title= 'Non-Sweets: (Fat to Sugar) to energy ratio',
    xaxis= dict(
        range=[0, 100],
        title="(Fat to Sugar) in g / 100g"
    ),
    yaxis=dict(
        range=[0, 5000],
        title="energy in J / 100g"
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='non-candy_fat-sugar_energy_ratio_scatter_limited')

In [83]:
high_energy = food_not_tagged_as_candy[food_not_tagged_as_candy["energy_100g"].astype(float) > 2000000]
#print(high_energy)

                code                                                url  \
35894  3256225050901  http://world-en.openfoodfacts.org/product/3256...   

                          creator   created_t      created_datetime  \
35894  openfoodfacts-contributors  1448645869  2015-11-27T17:37:49Z   

      last_modified_t last_modified_datetime                    product_name  \
35894      1479051222   2016-11-13T15:33:42Z  Goûters fourrés  chocolat noir   

      generic_name quantity                     packaging  \
35894          NaN    185 g  Sachet,Plastique,Etui,Carton   

                     packaging_tags   brands brands_tags categories  \
35894  sachet,plastique,etui,carton  U bio,U     u-bio,u        NaN   

      categories_tags categories_en origins origins_tags manufacturing_places  \
35894             NaN           NaN     NaN          NaN                  NaN   

      manufacturing_places_tags  \
35894                       NaN   

                                             

In [84]:
trace = go.Scatter(
    x = tagged_candies['fat/sugar'],
    y = tagged_candies["energy_100g"].astype(float),
    mode = 'markers'
)

layout = go.Layout(
    title= 'Sweets: (Fat to Sugar) to energy ratio',
    xaxis= dict(
        range=[0, 100],
        title="(Fat to Sugar) in g / 100g"
    ),
    yaxis=dict(
        #range=[0, 100],
        title="energy in J / 100g"
    )
)

data = [trace]

fig = go.Figure(data=data, layout=layout)
#py.iplot(fig, filename='candy_fat-sugar_energy_ratio_scatter')

In [85]:
candies_below_4_fat = tagged_candies[tagged_candies['fat_100g'].astype(float) < 4]

# printCategoryInfo(candies_below_4_fat)  # Jelly / Pudding

#TODO: split gained info into subgroups 
#(what differentiates jelly and pudding? Salt or sth?, then get stuff on other values (include the sugar side as well))


In [86]:
# <4 fat, <25 sugars, candy: pudding  (+ some haribo & mint)
sugars_40_to_60_below_4_fat = candies_below_4_fat[candies_below_4_fat["sugars_100g"].astype(float) <= 25]

#printCategoryInfo(candies_below_4_fat)

Product names:
('tirlibibi', 2)
('les', 2)
('morceaux', 2)
('mit', 2)
('vanilla', 2)
('mint', 2)
('party', 2)
('meringues', 2)
('turtles', 2)
('sahne', 2)
('vanille', 2)
('eucalyptus', 2)
('bonbons', 2)
('cerise', 2)
('friend', 2)
('nature', 2)
('dessert', 2)
("fisherman's", 2)
('coupe', 2)
('mini', 2)
('fruits', 2)
('goldbären', 2)
('frucht-kaltschale', 2)
('-', 2)
('schtroumpfs', 2)
('des', 2)
('yaourt', 2)
('à', 3)
('en', 3)
('unser', 3)
('pomme', 3)
('saveur', 3)
('schokolade', 3)
('délice', 4)
('de', 4)
('mix', 5)
('haribo', 5)
('menthe', 5)
('pudding', 8)
-----------
Categories_en:
('desserts,puddings', 11)
('snacks', 13)
('desserts', 23)
('snacks,confectioneries', 30)
('sugary', 44)
-----------
Categories_tags:
('en:desserts,en:puddings', 11)
('en:sugary-snacks', 13)
('en:desserts', 23)
('en:sugary-snacks,en:confectioneries', 30)
-----------
Categories:
('pudding', 2)
('confectionary', 3)
('confiserie', 4)
('desserts,puddings', 4)
('snacks,confectioneries', 4)
('puddings', 5)
('

In [87]:
# <4 fat, <25 sugars, non_candy: drinks, yogurts, fruits
sugars_25sugars_below_4_fat_noncandy = food_not_tagged_as_candy[food_not_tagged_as_candy["fat_100g"].astype(float) < 4]
sugars_25sugars_below_4_fat_noncandy = sugars_25sugars_below_4_fat_noncandy[sugars_25sugars_below_4_fat_noncandy["sugars_100g"].astype(float) < 25]

#printCategoryInfo(sugars_25sugars_below_4_fat_noncandy, 20)

In [88]:
# <4 fat, 40-60 sugars candy: Haribo
sugars_40_to_60_below_4_fat = candies_below_4_fat[candies_below_4_fat["sugars_100g"].astype(float) <= 60]
sugars_40_to_60_below_4_fat = sugars_40_to_60_below_4_fat[sugars_40_to_60_below_4_fat["sugars_100g"].astype(float) >= 40]

# printCategoryInfo(sugars_40_to_60_below_4_fat)

In [89]:
# <4 fat, 40-60 sugars non-candy: beverages, jelly, bonbons
sugars_40_to_60_below_4_fat_noncandy = food_not_tagged_as_candy[food_not_tagged_as_candy["fat_100g"].astype(float) < 4]
sugars_40_to_60_below_4_fat_noncandy = sugars_40_to_60_below_4_fat_noncandy[sugars_40_to_60_below_4_fat_noncandy["sugars_100g"].astype(float) <= 60]
sugars_40_to_60_below_4_fat_noncandy = sugars_40_to_60_below_4_fat_noncandy[sugars_40_to_60_below_4_fat_noncandy["sugars_100g"].astype(float) >= 40]

#printCategoryInfo(sugars_40_to_60_below_4_fat_noncandy, 20)

# seems like I missed a bunch of tags :xxxx might want to revisit that, then keep looking for clusters to work with

In [90]:
# 30-42 fat, 40-60 sugar  candy: chocolate
candies_at_x_sugar_fat = tagged_candies[tagged_candies['fat_100g'].astype(float) > 30]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['fat_100g'].astype(float) < 42]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['sugars_100g'].astype(float) > 40]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['sugars_100g'].astype(float) < 60]

#printCategoryInfo(candies_at_x_sugar_fat, 20)

In [91]:
# 30-42 fat, 40-60 sugar  non-candy: chocolate
candies_at_x_sugar_fat = food_not_tagged_as_candy[food_not_tagged_as_candy['fat_100g'].astype(float) > 30]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['fat_100g'].astype(float) < 42]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['sugars_100g'].astype(float) > 40]
candies_at_x_sugar_fat = candies_at_x_sugar_fat[candies_at_x_sugar_fat['sugars_100g'].astype(float) < 60]

#printCategoryInfo(candies_at_x_sugar_fat, 20)

In [92]:
### Machine Learning Stuff ###
from sklearn_pandas import DataFrameMapper

#pd.to_numeric(tagged_candies, errors='coerce')
#tagged_candies[['sugars_100g','fat_100g', 'salt_100g', 'energy_100g']] = tagged_candies[['sugars_100g','fat_100g', 'salt_100g', 'energy_100g']].apply(lambda x: pd.to_numeric(x, errors='coerce'))

# TODO: use .apply to split tags and find all candies, use sklean-pandas to start some machine learning stuffs

d:\Users\Odin\Anaconda3\lib\site-packages\pandas\core\frame.py:2378: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score


# 100 trees, seed for rnd-ness is 22
forest = RandomForestClassifier(n_estimators = 100, random_state = 22)

prediction_features = pd.read_csv(zf.open('food.csv'), parse_dates=True, dtype=object, delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8', 
                                  usecols=["sugars_100g", 
                                           "fat_100g",
                                           "salt_100g",
                                           "energy_100g",
                                           "categories_en",
                                          ])

# turn all objects (aside from categories) into floats
prediction_features = prediction_features.apply(pd.to_numeric, errors='ignore')

# remove columns with NA values
prediction_features = prediction_features.dropna()
# print(prediction_features)

# put labels in separate dataframe
labels_to_predict = prediction_features["categories_en"]
prediction_features.drop('categories_en', axis=1, inplace=True)

# turn labels_to_predict from a string-dataframe into a numbers-dataframe (indicating whether the word 'chocolate' is in categories with 1 or 0)
indeces = []
values = []
for index, value in labels_to_predict.iteritems():
    indeces.append(index)
    if 'chocolate' in value:
        values.append(1.0)
    else:
        values.append(0.0)

values = np.asarray(values)  # turn values into np array
labels_dict = {'is_chocolate' : pd.Series(values, indeces)} 

labels_to_predict = pd.DataFrame(labels_dict)

for index, value in labels_to_predict['is_chocolate'].iteritems():
    pass#print(value)

# TODO: need to split this into 3 sets: training, validation(to tweak), test
X_train, X_test, y_train, y_test = train_test_split(prediction_features, labels_to_predict, random_state=22)

In [94]:
# build tree (fit) and get cross val score using training data

X_train_len = len(X_train["fat_100g"].values)
y_train_len = len(y_train["is_chocolate"].values)

fat_data = X_train["fat_100g"].values.reshape(X_train_len, 1)
sugars_data = X_train["sugars_100g"].values.reshape(X_train_len, 1)
salt_data = X_train["salt_100g"].values.reshape(X_train_len, 1)
energy_data = X_train["energy_100g"].values.reshape(X_train_len, 1)

# combine training data and flatten label-column to list
train_data = np.hstack([fat_data, sugars_data, salt_data, energy_data])
final_labels_train = np.ravel(y_train)

forest = forest.fit(train_data, final_labels_train)

# determine cross validation score
scores = cross_val_score(forest, train_data, final_labels_train, scoring='accuracy', cv = 5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.98 (+/- 0.01)


In [99]:
# get accuracy based on test data

X_test_len = len(X_test["fat_100g"].values)

fat_test = X_test["fat_100g"].values.reshape(X_test_len, 1)
sugars_test = X_test["sugars_100g"].values.reshape(X_test_len, 1)
salt_test = X_test["salt_100g"].values.reshape(X_test_len, 1)
energy_test = X_test["energy_100g"].values.reshape(X_test_len, 1)

final_data_test = np.hstack([fat_test, sugars_test, salt_test, energy_test])
final_labels_test = np.ravel(y_test)

y_predicted = forest.predict(final_data_test)
for i in y_predicted:
    print(i)
y_accurancy = np.mean(final_labels_test == y_predicted) * 100
print("Accuracy of test data: {0:.1f}%".format(y_accurancy))

print(fat_test)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [105]:
def predict_chocolate(df):
    print("removing columns with NaN, might prevent aligning original df with predictions; pls remove NaN columns ahead of time")
    # turn everything you can into floats and remove columns with NA values
    df = df.apply(pd.to_numeric, errors='ignore')
    df = df.dropna(subset=["fat_100g", "sugars_100g", "salt_100g", "energy_100g"])

    length = len(df["fat_100g"].values)
    
    fat_data = df["fat_100g"].values.reshape(length, 1)
    sugars_data = df["sugars_100g"].values.reshape(length, 1)
    salt_data = df["salt_100g"].values.reshape(length, 1)
    energy_data = df["energy_100g"].values.reshape(length, 1)
    
    # combine training data and flatten label-column to list
    final_data = np.hstack([fat_data, sugars_data, salt_data, energy_data])
        
    
    return forest.predict(final_data)

#print(predict_chocolate(food))

cleansed_candies = tagged_candies.apply(pd.to_numeric, errors='ignore')
cleansed_candies = cleansed_candies.dropna(subset=["fat_100g", "sugars_100g", "salt_100g", "energy_100g"])

acc = 0
for i in (predict_chocolate(cleansed_candies)):
    acc += i
    
print("predicts: ", acc)

acc = 0
for index, value in cleansed_candies['categories_en'].iteritems():
    if "Choc" in value:
        acc += 1
        
print("chcocs: ", acc)

removing columns with NaN, might prevent aligning original df with predictions; pls remove NaN columns ahead of time
predicts:  60.0
chcocs:  294
